In this notebook, I am collecting everything one needs to run densepose within the google colab environment.

First, we'll mount our google drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Next, we'll install Anaconda2 to then use it to install caffe2, install the COCO API and finally densepose. For the latter, we have to make sure we are using the correct & latest "env.py" from the detectron utils. Finally, we'll fetch the densepose data.

In [2]:
import os

!wget -c https://repo.continuum.io/archive/Anaconda2-5.3.0-Linux-x86_64.sh
!chmod +x Anaconda2-5.3.0-Linux-x86_64.sh
!bash ./Anaconda2-5.3.0-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python2.7/site-packages/')

!conda install -y pytorch-nightly -c pytorch
!pip install argparse
!pip install pyhamcrest
!pip install protobuf
!pip install future
!pip install pydot
!pip install cython scikit-image easydict

!python2 -c 'from caffe2.python import workspace; print(workspace.NumCudaDevices())'
!python2 -c 'from caffe2.python import core' 2>/dev/null && echo "Success" || echo "Failure"

COCOAPI='/usr/local/cocoapi/'
!git clone https://github.com/cocodataset/cocoapi.git $COCOAPI
  
os.chdir('/usr/local/cocoapi/PythonAPI/')
!python2 setup.py install --user

DENSEPOSE='/usr/local/densepose/'
!git clone https://github.com/facebookresearch/densepose $DENSEPOSE
os.chdir('/usr/local/densepose/detectron/utils/')
!rm env.py
!wget https://raw.githubusercontent.com/facebookresearch/Detectron/master/detectron/utils/env.py
!pip install -r $DENSEPOSE/requirements.txt
os.chdir('/usr/local/densepose/')
!make
!python $DENSEPOSE/detectron/tests/test_spatial_narrow_as_op.py

os.chdir('/usr/local/densepose/DensePoseData/')
!bash get_densepose_uv.sh
!bash get_eval_data.sh

--2018-11-24 13:30:12--  https://repo.continuum.io/archive/Anaconda2-5.3.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.16.18.10, 104.16.19.10, 2606:4700::6810:130a, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.16.18.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 647650387 (618M) [application/x-sh]
Saving to: ‘Anaconda2-5.3.0-Linux-x86_64.sh’

Anaconda2-5.3.0-Lin 100%[===================>] 617.65M   163MB/s    in 3.8s    

2018-11-24 13:30:16 (164 MB/s) - ‘Anaconda2-5.3.0-Linux-x86_64.sh’ saved [647650387/647650387]

PREFIX=/usr/local
reinstalling: python-2.7.15-h1571d57_0 ...
Python 2.7.15 :: Anaconda, Inc.
reinstalling: blas-1.0-mkl ...
reinstalling: ca-certificates-2018.03.07-0 ...
reinstalling: conda-env-2.6.0-1 ...
reinstalling: intel-openmp-2019.0-118 ...
reinstalling: libgcc-ng-8.2.0-hdf63c60_1 ...
reinstalling: libgfortran-ng-7.3.0-hdf63c60_0 ...
reinstalling: libstdcxx-ng-8.2.0-hdf63c60_1 ...
reinstalling: 

Now we're ready to infer the positions of bodies in images:

In [0]:
os.chdir('/usr/local/densepose/')
!python2 tools/infer_simple.py \
    --cfg configs/DensePose_ResNet101_FPN_s1x-e2e.yaml \
    --output-dir DensePoseData/infer_out/ \
    --image-ext jpg \
    --wts https://s3.amazonaws.com/densepose/DensePose_ResNet101_FPN_s1x-e2e.pkl \
    /content/gdrive/My\ Drive/skopdaube/benjaminDanceTest/

In [0]:
os.chdir('/usr/local/densepose/')
!python2 tools/infer_simple.py \
    --cfg configs/DensePose_ResNet101_FPN_s1x-e2e.yaml \
    --output-dir DensePoseData/infer_out/ \
    --image-ext jpg \
    --wts https://s3.amazonaws.com/densepose/DensePose_ResNet101_FPN_s1x-e2e.pkl \
    /content/gdrive/My\ Drive/skopdaube/benjDance720/

Copy the visualisations of the detected bodies into our drive

In [0]:
cp DensePoseData/infer_out/*IUV.png /content/gdrive/My\ Drive/skopdaube/testOut/

Here I am trying to use trrahul's code that is supposed to work directly on video files instead of the single frames. So far, I failed to install ffmpeg though.

In [0]:

cd /usr/local/densepose/tools/
!wget https://raw.githubusercontent.com/trrahul/densepose-video/master/infer_vid.py
!wget https://raw.githubusercontent.com/trrahul/densepose-video/master/vis.py

In [0]:
!python2 tools/infer_vid.py \
    --cfg configs/DensePose_ResNet101_FPN_s1x-e2e.yaml \
    --output-dir DensePoseData/infer_out/ \
    --wts https://s3.amazonaws.com/densepose/DensePose_ResNet101_FPN_s1x-e2e.pkl \
    --input-file /content/gdrive/My\ Drive/skopdaube/videoTest/test1.mov
